## Salaries Dataset Cleaning and Aggregation

##### This notebooks highlights case study analysis on salaries dataset gathered from Kaggle, and will be used to highlight:
- Data Cleaning Techniques
- Data Aggregations and Pivot Tables
- Exploratory Data Analysis (EDA)

In [90]:
# import necessary libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Simple 'ETL' Pipeline Class for the Dataset

In [ ]:
# a basic data pipeline class that loads and deletes empty cols
class Pipeline:

    def __init__(self) -> None:
        self.data = None

    # sql query method
    def query(self, query: any):
        pass
    
    # loads csv file
    def load_csv(self, src: str):
        
        try:
            self.data = pd.read_csv(src)
            return self.data
        except ValueError as e:
            return "The DataFrame is empty."

    # drops empty columns
    def drop_cols(self, cols: list):
        self.data = self.data.drop(columns=cols, axis=1)
        return self.data

    def drop_rows(self, row: list):
        self.data = self.data.drop(rows=row)
        return self.data
    
    # fill values, drop missing values
    def fill_values(self, values: int):
        self.data = self.data.fillna(value=values)
        return self.data
    
    def drop_missing(self):
        self.data = self.data.dropna()
        return self.data

In [92]:
data = Pipeline()

df = data.load_csv("..\data\Salaries.csv")

C:\Users\LEI\AppData\Local\Temp\ipykernel_21784\1599444425.py:15: DtypeWarning: Columns (3,4,5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(src)


In [93]:
df.columns

Index(['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay',
       'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Notes', 'Agency',
       'Status'],
      dtype='object')

In [94]:
# renaming columns
headers = [
    "id", "employee-name", "job-title", "base-pay", "overtime-pay", "other-pay",
    "benefits", "total-pay", "total-pay-benefits", "year", "notes", "agenecy", "status"
]

df.columns = headers

In [95]:
df.describe(include='all')

,id,employee-name,job-title,base-pay,overtime-pay,other-pay,benefits,total-pay,total-pay-benefits,year,notes,agenecy,status
count,148654.000000,148654,148654,148049.0,148654.0,148654.0,112495.0,148654.000000,148654.000000,148654.000000,0.0,148654,38119
unique,NaN,110811,2159,109900.0,66555.0,84968.0,99635.0,NaN,NaN,NaN,NaN,1,2
top,NaN,Kevin Lee,Transit Operator,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,San Francisco,FT
freq,NaN,13,7036,875.0,66103.0,35218.0,1053.0,NaN,NaN,NaN,NaN,148654,22334
mean,74327.500000,NaN,NaN,NaN,NaN,NaN,NaN,74768.321972,93692.554811,2012.522643,NaN,NaN,NaN
std,42912.857795,NaN,NaN,NaN,NaN,NaN,NaN,50517.005274,62793.533483,1.117538,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,-618.130000,-618.130000,2011.000000,NaN,NaN,NaN
25%,37164.250000,NaN,NaN,NaN,NaN,NaN,NaN,36168.995000,44065.650000,2012.000000,NaN,NaN,NaN
50%,74327.500000,NaN,NaN,NaN,NaN,NaN,NaN,71426.610000,92404.090000,2013.000000,NaN,NaN,NaN
75%,111490.750000,NaN,NaN,NaN,NaN,NaN,NaN,105839.135000,132876.450000,2014.000000,NaN,NaN,NaN


In [96]:
# summary statistics on all numerical columns
df.describe()

,id,total-pay,total-pay-benefits,year,notes
count,148654.000000,148654.000000,148654.000000,148654.000000,0.0
mean,74327.500000,74768.321972,93692.554811,2012.522643,NaN
std,42912.857795,50517.005274,62793.533483,1.117538,NaN
min,1.000000,-618.130000,-618.130000,2011.000000,NaN
25%,37164.250000,36168.995000,44065.650000,2012.000000,NaN
50%,74327.500000,71426.610000,92404.090000,2013.000000,NaN
75%,111490.750000,105839.135000,132876.450000,2014.000000,NaN
max,148654.000000,567595.430000,567595.430000,2014.000000,NaN


In [97]:
# dropping unnecessary/empty columns
data.drop_cols(['benefits', 'notes', 'status'])

,id,employee-name,job-title,base-pay,overtime-pay,other-pay,total-pay,total-pay-benefits,year,agenecy
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,567595.43,567595.43,2011,San Francisco
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,538909.28,538909.28,2011,San Francisco
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,335279.91,335279.91,2011,San Francisco
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,332343.61,332343.61,2011,San Francisco
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,326373.19,326373.19,2011,San Francisco
...,...,...,...,...,...,...,...,...,...,...
148649,148650,Roy I Tillery,Custodian,0.00,0.00,0.00,0.00,0.00,2014,San Francisco
148650,148651,Not provided,Not provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,San Francisco
148651,148652,Not provided,Not provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,San Francisco
148652,148653,Not provided,Not provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,San Francisco
